<a href="https://colab.research.google.com/github/anamilanezi/ebac-dados/blob/main/projetos/em16-analise-exploratoria-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Análise Exploratória de Dados de Logística I
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução ao Kaggle;</li>
  <li>Introdução ao problema de negócios;</li>
  <li>Exploração de dados.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para a construção da sua própria análise exploratória de dados. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/).

---

# **Análise Exploratória de Dados de Logística**

## 1\. Contexto

Neste problema, iremos utilizar os dados sintetizados de fontes públicas (IBGE, IPEA, etc.) que são representativos dos desafios que a startup Loggi enfrenta no dia a dia, como otimização das rotas de entrega, alocação de entregas nos veículos da frota com capacidade limitada, etc. O dado bruto é um arquivo do tipo JSON com uma lista de instâncias de entregas. Cada instância representa um conjunto de entregas que devem ser realizadas pelos veículos do hub regional.

Os dados estão organizados da seguinte forma:
 - **name**: uma `string` com o nome único da instância;
 - **region**: uma `string` com o nome único da região do **hub**;
 - **origin**: um `dict` com a latitude e longitude da região do **hub**;
 - **vehicle_capacity**: um `int` com a soma da capacidade de carga dos **veículos** do **hub**;
 - **deliveries**: uma `list` de `dict` com as **entregas** que devem ser realizadas:
    - **id**: uma `string` com o id único da **entrega**;
    - **point**: um `dict` com a latitude e longitude da **entrega**;
    - **size**: um `int` com o tamanho ou a carga que a **entrega** ocupa no **veículo**.


## 2\. Pacotes e bibliotecas

In [2]:
import json
import pandas as pd
from pprint import pprint

## 3\. Exploração de dados

## 3.1. Coleta

Obtendo os dados brutos no formato `.json`:

In [3]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json 

In [4]:
with open('deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

In [5]:
# Formato do dado coletado:
type(data)

list

In [6]:
# Quantidade de itens da lista
len(data)

199

In [7]:
# Atribuindo o primeiro item da lista a uma variável para verificar sua organização
primeiro_item = data[0]

# Cada item da lista corresponde a um dicionário aninhado, que possui os dados de diferentes formatos:
print(primeiro_item.keys())
print(type(primeiro_item['name']))
print(type(primeiro_item['region']))
print(type(primeiro_item['origin']))
print(type(primeiro_item['vehicle_capacity']))
print(type(primeiro_item['deliveries']))

dict_keys(['name', 'region', 'origin', 'vehicle_capacity', 'deliveries'])
<class 'str'>
<class 'str'>
<class 'dict'>
<class 'int'>
<class 'list'>


In [8]:
print(len(primeiro_item['deliveries']))

4588


In [9]:
pprint(primeiro_item['deliveries'][0])

{'id': '313483a19d2f8d65cd5024c8d215cfbd',
 'point': {'lat': -15.848929154862294, 'lng': -48.11618888384239},
 'size': 9}


## 3.2. Wrangling

In [10]:
deliveries_df = pd.DataFrame(data)

In [11]:
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


As colunas `origin` e `deliveries` possuem dados em formato de dicionário e lista, respectivamente, e isso exige alguns métodos da biblioteca pandas para descompactar essas informações.

Iniciando com a coluna `origin`, vamos criar um segundo dataframe utilizando o método `.json_normalize`, numa operação conhecia como `flatten` que transforma cada chave do JSON em uma nova coluna, separando as informações que estão aninhadas como dicionário.

In [12]:
hub_origin_df = pd.json_normalize(deliveries_df["origin"])
hub_origin_df.head()

,lng,lat
0,-48.054989,-15.838145
1,-48.054989,-15.838145
2,-48.054989,-15.838145
3,-47.893662,-15.805118
4,-48.054989,-15.838145


Com os dados separados em colunas, é possível juntar os dois dataframes com o método merge utilizando o index das linhas como chave para uní-los.

In [13]:
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df, how='inner', left_index=True, right_index=True)

In [14]:
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries,lng,lat
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145


Com os dados unidos em um só dataframe, e as novas colunas de lng e lat, vamos remover a coluna `origin` contendo os dados no dicionário aninhado e reordenar as colunas.

In [15]:
deliveries_df = deliveries_df.drop("origin", axis=1)
deliveries_df = deliveries_df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]
deliveries_df.head()

,name,region,lng,lat,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,-48.054989,-15.838145,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,-48.054989,-15.838145,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,-47.893662,-15.805118,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,-48.054989,-15.838145,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


Visto que em deliveries também existem informações referentes à latitude e longitude do ponto de entrega, vamos renomear as colunas recém criadas para especificar que se referem ao hub de origem:

In [16]:
deliveries_df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"}, inplace=True)
deliveries_df.head()

,name,region,hub_lng,hub_lat,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,-48.054989,-15.838145,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,-48.054989,-15.838145,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,-47.893662,-15.805118,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,-48.054989,-15.838145,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


Agora vamos fazer o tratamento da coluna deliveries, onde cada linha contém uma lista de JSON (que corresponde ao formato de um dicionário python). Para isso também vamos criar um novo dataframe a partir dessa coluna, utilizando o método `explode`, que transforma cada elemento da lista em uma nova linha do dataframe. Verifica-se que o dataframe criado possui 636149 linhas, e elas mantém o index da linha do dataframe de origem.

In [17]:
deliveries_exploded_df = deliveries_df[["deliveries"]].explode("deliveries")
print(len(deliveries_exploded_df))

636149


In [18]:
deliveries_exploded_df.head()

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."


In [19]:
deliveries_exploded_df.tail()

,deliveries
198,"{'id': '21693bf442ac5890adbdf2648c12881a', 'po..."
198,"{'id': '7aaa35088b37b6e542c4cd69663a7ebf', 'po..."
198,"{'id': '60c00d5390da4f28167439cd9c566703', 'po..."
198,"{'id': '51f456963785e7381243ff7baf7efd06', 'po..."
198,"{'id': 'b7078c815198669e2aab4336e94c0bb8', 'po..."


Agora cada linha será primeiramente transformada em um dataframe com o método apply sobre a coluna, selecionando as informações através da chave do cicionário das informações referentes à latitude, longitude e tamanho da carga (o id não será utilizado). Com cada informação separada em um dataframe, eles são então concatenados em `deliveries_normalized_df`.

In [ ]:
deliveries_normalized_df = pd.concat([
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["size"])).rename(columns={"deliveries": "delivery_size"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)
deliveries_normalized_df.head()